Basic LOSO
----------
This covers a fundamental use-case for DN<sup>3</sup>. Here, the TIDNet architecture from *Kostas & Rudzicz 2020
(under review)* is evaluated using the BCI competition IV dataset 2a four-class motor imagery dataset in a
*"leave one subject out"* training procedure. This means, for each person in the dataset, the remaining subjects are
used to develop a classifier, and performance is measured using

Here we assume that the original *.gdf* files that comprise the dataset were
[downloaded from the original source](<http://www.bbci.de/competition/iv/>), then were organized into a
subdirectory per person containing the *T* (training) and *E* (evaluation) sessions. In this common format, we can use
the following configuration file (let's call it `bci_iv_config.yml`) to easily import the dataset

```yaml
DN3:
  datasets:
    - bci_2a
  epochs: 5
  batch_size: 60

bci_2a:
  name: "BCI Competition IV - dataset 2a"
  toplevel: /path/to/the/toplevel/folder
  tmin: -0.5
  tlen: 4.5
  picks:
    - eeg
    - eog
  events:
    left_hand: 769
    right_hand: 770
    foot: 771
    tongue: 772

```

Notice that in addition to the dataset, we include some general experimental configuration parameters in the DN3
section. These are *not mandatory*, its just a convenient place to keep track of variables like this, so that our code
doesn't have to change.

In [1]:
from dn3.configuratron import ExperimentConfig
from dn3.trainable.processes import StandardClassification
from dn3.trainable.models import TIDNet

# Since we are doing a lot of loading, this is nice to suppress some tedious information
import mne
mne.set_log_level(False)

First things first, we use `ExperimentConfig`, and the subsequently constructed `DatasetConfig` to rapidly construct
our `dataset`.

In [2]:
config_filename = 'bci_iv_config.yml'
use_gpu = False
experiment = ExperimentConfig(config_filename)
dataset = experiment.datasets['bci_2a']

dataset = dataset.auto_construct_dataset()

FileNotFoundError: [Errno 2] No such file or directory: 'bci_iv_config.yml'

Now we can simply use the `.loso()` method to loop over each person in the dataset. Good practice would see that any
validation data (data held out from the *training set* that is evaluated to track sensitivity to the training data
alone), is a similarly set aside person. So the default in DN<sup>3</sup> is to provide a unique validation
person for each testing person.

Let's create a function that makes a new model for each set of training people and a trainable process for
`StandardClassification`.

In [ ]:
def make_model_and_process():
    tidnet = TIDNet.from_dataset(dataset)
    return StandardClassification(tidnet)

That's it! We use a helper that initializes a TIDNet from any `Dataset/Thinker/EpochedRecording` and wrap it with a
`StandardClassification` process. Invoking this process will train the classifier. Looking through `trainable.processes`
will show you what other processes can be invoked (perhaps on the same model, e.g. if you wanted to fine-tune this
model later).

This is really all you need, with a handful of lines of code, state-of-the-art results. Just loop through the `.loso()`
method, fit the classifier, then check the test results.

In [ ]:
results = dict()
for training, validation_thinker, test_thinker in dataset.loso():
    process = make_model_and_process()

    process.fit(training_dataset=training, validation_dataset=validation_thinker, epochs=experiment.epochs, batch_size=experiment.batch_size)

    performance = process.evaluate(test_thinker)
    results[test_thinker.person_id] = performance

print(results)
print("Average accuracy: {:.2%}".format(sum(results.values())/len(results)))